# Welcome to the Lyrics Generator for Genres! 

You can use this notebook to finetune the GPT-2 Language Model on datasets containing songs from different genres. Pick a genre and see what the generator has in store. The data comes from MetroLyrics.

Would you rather use your own songs? Go check out the [Lyrics Scraper](https://github.com/pieter-cappon/State-of-the-Art-Technology/blob/master/lyricscollector.ipynb). Remember that when using your own songs it will take longer before you can generate!

---

In [ ]:
#@markdown If anything goes wrong with the notebook, use the killswitch to restart. To prevent accidental clicking you have to run this cell first.
#@markdown You will then see a button, once you click the button there is no going back..
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Killswitch")
output = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    !kill -9 -1

button.on_click(on_button_clicked)
display(button, output)


Button(description='Killswitch', style=ButtonStyle())

Output()

In [ ]:
#@markdown First off we need to import some libraries and connect with the datasets that we are going to use for training. Run this cell.

# importing libraries
import os
import io
import requests
import pandas as pd
import csv
# set Tensorflow to 1.x as 2.x would lead to problems with gpt_2_simple.

%tensorflow_version 1.x

try:
  import gpt_2_simple as gpt2
except:
  !pip3 -q install gpt-2-simple
  import gpt_2_simple as gpt2

for model_name in ["124M","355M","774M"]:  # Choose from ["124M","355M","774M"]
    gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/124M/

!git clone https://github.com/pieter-cappon/State-of-the-Art-Technology.git

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 391Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 83.4Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 323Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 192Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 657Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 93.7Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 112Mit/s]                                                       
Fetching checkpoint: 1.05Mit [00:00, 268Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 73.9Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 349Mit/s]                                           

Cloning into 'State-of-the-Art-Technology'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 37 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (37/37), done.


In [ ]:
#@title Select the genre of your choice and the model you want to finetune. 

# set some hyperparameters and start a session

genre = "metal" #@param ["pop", "metal", "hiphop", "country"]

#@markdown Remember: the higher the model the longer it takes to generate, 774M might not always work

learning_rate = 0.0001
optimizer = 'adam' 
model_name = "124M" #@param ["774M", "355M", "124M"] 
sess = gpt2.start_tf_sess()

# Retrain model with new data
gpt2.finetune(sess,
              'State-of-the-Art-Technology/{}-lyrics.csv'.format(genre),
              model_name=model_name,
              sample_every=50,
              save_every=50,
              print_every=10,
              learning_rate=learning_rate,
              restore_from='latest',
              steps=500)   # steps is max number of training steps

#@markdown Once you're happy with your choice, run the cell! Relax while the model works for you.
#@markdown You can see how it's doing as it prints out some samples once in a while.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00,  5.18it/s]

Loading dataset...


dataset has 3242929 tokens
Training...
[10 | 50.34] loss=3.09 avg=3.09
[20 | 93.50] loss=3.01 avg=3.05
[30 | 136.71] loss=3.30 avg=3.14
[40 | 179.90] loss=3.21 avg=3.15
[50 | 223.11] loss=3.28 avg=3.18
Saving checkpoint/run1/model-50
======== SAMPLE 1 ========
D!
They won't do nothing for us when the earth is falling apart.
We don't need help for this to happen!
No one seems to know what it means anymore!
What will keep us going for so long?
For all we need to become what we are!
For all we need to be!
We were saved!
We will win!
And in one fell swoop, we will triumph!
For all we need to become what we are!
For all we need to become!
And in one fell swoop, we will triumph
For all we need to become what we are!
For all we need to become what we are!
For all we need to be!<|endoftext|>
<|startoftext|>Burden
So strong are you, your life is so thin and you're bleeding
Your blood and blood is so weak and bleeding
It won't stop, it will take all its strength
They will cut your heart by force

#Let's generate some lyrics

In [ ]:

#@markdown You can still pick out or change a few things, don't worry, you can rerun this cell if it's not yet what you want.

#@markdown The temperature controls the degree of surprise, so: if you want to spice it up, crank it up. This is useful when your generated text is a bit repetitive.
temperature = 0.8 #@param {type:"slider", min: 0, max:1, step:0.1}



#@markdown Choose how long your texts are. <u>1024</u> is the maximum.
length =  200#@param {type: "number"}

#@markdown Already inspired and want to see what the generator can make of your initial idea? 
#@markdown Prime the model with some of your lyrics!

idea = 'This is a song about'  #@param {type: "string"}
from datetime import datetime
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

results = (gpt2.generate(sess,           
              length=length,
              destination_path=gen_file,
              temperature=temperature,
              prefix=idea,
              nsamples=5,
              batch_size=5,
              return_as_list=True,
              ))


**Now we can view the results**

In [ ]:
for res in results:
  print(res)
  print('\n -------//------ \n')

A generation lost
Waiting for it
Now we'll be free
Wet and dry
Wet and dry
Soaked and cold
Soaked and cold
Wet and dry
Soaked and cold
Wet and dry
As we get older
The length of our lives
Will be forever longer
Its time to jump
Wet and dry
Wet and dry<|endoftext|>
<|startoftext|>You are so much more than I can bear
You are so much more than I can miss
You are so much more than I can miss
You will be where we are tonight
You will be where we are tonight
You will be where we are tonight
You will be where we are tonight
You will be where we are tonight
You will be where we are tonight
You will be where we are tonight
You will be where we are tonight
You will be where we are tonight
You will be tonight
You will be tonight
You will be tonight
You will be tonight
You will be tonight
You will be tonight
You will be tonight<|endoftext|>
<|startoftext|>The type of you that I've been seeing
Is so bad that I can't believe
That you truly want me
I have no respect for you
You have no respect for me


#Happy with your texts?
You can download a text file by looking in the file manager in the sidebar and selecting the one with the right date and time.